# How does affordability vary across Illinois counties?

#### Methodology
1. Extract, combine, and clean data
2. Merge dataframes
3. Calculate home price to income ratio
4. Plot final findings 

### Import Libraries and Data


In [9]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import fredapi as fa
import seaborn as sns
api_key = 'api-key'

fred = fa.Fred(api_key=api_key)

counties_data = pd.read_csv(r'/csv_files/Counties.csv')


# Create an empty list to store DataFrames
dataframes = []

# Loop through each county and fetch data from FRED
for index, row in counties_data.iterrows():
    county_name = row['county_name']
    series_id = row['series_id']

    try:
        data = fred.get_series(series_id)
        data = data.to_frame(name='median_price')
        data['median_price'] = data['median_price'].astype(int) #converts median listing price to int
        data['county_name'] = county_name
        data['year'] = data.index.year
        mask = data['year'].apply(lambda x: x < 2023) #filter data by year to match other datasets
        filtered_data = data[mask]
        dataframes.append(filtered_data)
    except Exception as e:
        print(f"Error retrieving data for {county_name}: {e}")

# Combine DataFrames
combined_df = pd.concat(dataframes)
# Aggregate monthly data to yearly
combined_df = combined_df.groupby(['county_name','year'])['median_price'].mean().reset_index()
combined_df.rename(columns={'index': 'Date'}, inplace=True) #readjust column after resetting


print(combined_df)


# Create an empty list to store DataFrames
income_dataframes = []

# Loop through county data and fetch data from FRED
for index, row in counties_data.iterrows():
    county_name = row['county_name']
    household_income = row['hh_income']

    try:
        income_data = fred.get_series(household_income, start='2016-01-01')
        income_data_df = income_data.to_frame(name='household_income')  # Ensure data is in a DataFrame
        income_data_df = income_data_df[income_data_df.index >= pd.to_datetime('2016-01-01')]

        income_data_df['household_income'] = income_data_df['household_income'].astype(int) #converts to int after fetching
        income_data_df['county_name'] = county_name
        income_data_df['year'] = income_data_df.index.year

        income_dataframes.append(income_data_df)
    except Exception as e:
        print(f"Error retrieving data for {county_name}: {e}")

# Combine DataFrames
combined_income_df = pd.concat(income_dataframes)
combined_income_df = combined_income_df.reset_index()
combined_income_df.rename(columns={'index': 'Date'}, inplace=True)


print(combined_income_df)

              county_name  year   median_price
0        Adams County, IL  2016  118383.333333
1        Adams County, IL  2017  124735.416667
2        Adams County, IL  2018  130712.500000
3        Adams County, IL  2019  135573.250000
4        Adams County, IL  2020  142675.000000
..                    ...   ...            ...
226  Winnebago County, IL  2018  127492.083333
227  Winnebago County, IL  2019  131141.750000
228  Winnebago County, IL  2020  137202.750000
229  Winnebago County, IL  2021  129516.666667
230  Winnebago County, IL  2022  145550.000000

[231 rows x 3 columns]
          Date  household_income           county_name  year
0   2016-01-01             51130      Adams County, IL  2016
1   2017-01-01             49088      Adams County, IL  2017
2   2018-01-01             55174      Adams County, IL  2018
3   2019-01-01             55942      Adams County, IL  2019
4   2020-01-01             57434      Adams County, IL  2020
..         ...               ...              

### Merge Data


In [10]:
merged_data = pd.merge(combined_df, combined_income_df, on=['year', 'county_name'], how='outer')
merged_data = merged_data.drop('Date', axis=1)

### Calculate Home Price to Income Ratio 

In [11]:
merged_data['pi_ratio'] = merged_data['median_price'] / merged_data['household_income']
print(merged_data)

               county_name  year   median_price  household_income  pi_ratio
0         Adams County, IL  2016  118383.333333             51130  2.315340
1     Champaign County, IL  2016  150237.500000             51032  2.943986
2         Coles County, IL  2016   85483.333333             42066  2.032124
3          Cook County, IL  2016  271391.666667             60025  4.521311
4       De Kalb County, IL  2016  169524.666667             59586  2.845042
..                     ...   ...            ...               ...       ...
226   Vermilion County, IL  2022  101027.083333             52083  1.939732
227   Whiteside County, IL  2022  119039.500000             62038  1.918816
228        Will County, IL  2022  349919.000000             97076  3.604588
229  Williamson County, IL  2022  132104.166667             64152  2.059237
230   Winnebago County, IL  2022  145550.000000             62557  2.326678

[231 rows x 5 columns]


### Find Top 5 Most and Least Affordable Counties 

Calculate the mean Home Price to Income Ratio and filter for the highest and lowest 5 counties. 

In [ ]:
# Calculate average PIR per county
avg_pir = merged_data.groupby('county_name')['pi_ratio'].mean()
# Get top 5 counties by average PIR
least_affordable_counties = avg_pir.sort_values(ascending=False).head(5).index
# Filter data for top 5 counties
least_affordable_data = merged_data[merged_data['county_name'].isin(least_affordable_counties)]
# Get bottom 5 counties by average PIR
most_affordable_counties = avg_pir.sort_values(ascending=True).head(5).index
# Filter data for bottom 5 counties
most_affordable_data = merged_data[merged_data['county_name'].isin(most_affordable_counties)]

Combine variables using pandas concatenate function

In [13]:
# Combine counties with highest and lowest average pi ratios
combined_affordability = pd.concat([least_affordable_data, most_affordable_data], ignore_index=True)

print(combined_affordability)

             county_name  year   median_price  household_income  pi_ratio
0        Cook County, IL  2016  271391.666667             60025  4.521311
1     Du Page County, IL  2016  391453.166667             84999  4.605386
2        Kane County, IL  2016  314056.500000             73684  4.262208
3        Lake County, IL  2016  375686.666667             83344  4.507663
4        Will County, IL  2016  261900.000000             81640  3.207986
..                   ...   ...            ...               ...       ...
65      Coles County, IL  2022   87952.083333             57712  1.523983
66      Henry County, IL  2022  122535.333333             70517  1.737671
67   Macoupin County, IL  2022   95970.833333             69880  1.373366
68  Vermilion County, IL  2022  101027.083333             52083  1.939732
69  Whiteside County, IL  2022  119039.500000             62038  1.918816

[70 rows x 5 columns]


### Plot Home Price to Income Ratio by County

In [ ]:
# Create the line plot
sns.lineplot(x="year", y="pi_ratio", hue="county_name", data=combined_affordability)
plt.xlabel("Year")
plt.ylabel("Home Price-to-Income Ratio")
plt.title("Home Price-to-Income Ratio by County Over Time")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

![Home Price to Income Ratio by County over Time](images\Home-PIR-Counties-Over-Time.png)